In [ ]:
import numpy as np
import pandas as pd
import os
import urllib
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load MNIST dataset from 
# https://www.python-course.eu/data/mnist/
train = pd.read_csv('https://www.python-course.eu/data/mnist/mnist_train.csv',
                    header=None)
test = pd.read_csv('https://www.python-course.eu/data/mnist/mnist_test.csv',
                    header=None)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.head()

In [ ]:
train[0].value_counts().sort_index()

In [ ]:
train_target = train[0]
train_image = train.drop([0], axis=1)
print(train_image.shape)

In [ ]:
index = 12345
image = train_image.loc[index].to_numpy()
image = image.reshape([28, 28])
plt.imshow(image, cmap=matplotlib.cm.binary)

In [ ]:
train_target[index]

In [ ]:
def plot_digit(data):
    image = data.to_numpy().reshape([28, 28])
    plt.imshow(image,
               cmap=matplotlib.cm.binary)

In [ ]:
plot_digit(train_image.loc[12345])

In [ ]:
def plot_digits(instances, images_per_row=10):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = matplotlib.cm.binary)
    plt.axis("off")

In [ ]:
plt.figure(figsize=(9,9))
example_images = np.r_[train_image[:30]]
plot_digits(example_images, images_per_row=10)

## Binary Classifier
Before building a classifier that can detect all 10 classes, let's build a classifier to identify image 5.

In [ ]:
train_target_5 = (train_target == 5)

In [ ]:
# Transform test set
test_target = test[0]
test_image = test.drop([0], axis=1)
test_target_5 = (test_target == 5)

In [ ]:
# Apply logistic regression model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# model.fit(train_image, train_target_5)

In [ ]:
# Apply the model to predict for an image
index = 1
image = train_image.loc[[index]]
plot_digit(image)

In [ ]:
model.predict(image)

In [ ]:
# What is the overall accuracy over the training set?
train_prediction = model.predict(train_image)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(train_target_5, train_prediction)

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision_score(train_target_5, train_prediction)

In [ ]:
recall_score(train_target_5, train_prediction)

## Multiclass Classification

In [ ]:
datasize = 1000
train_image_small = train_image[:datasize]
train_target_small = train_target[:datasize]
model2 = LogisticRegression(multi_class='ovr')
model2.fit(train_image_small, train_target_small)

In [ ]:
train_prediction_small = model2.predict(train_image_small)
confusion_matrix(train_target_small, train_prediction_small)

In [ ]:
test_prediction = model2.predict(test_image)
confusion_matrix(test_target, test_prediction)

## Data Augmentation

In [ ]:
from scipy.ndimage.interpolation import shift

def shift_image(image, dx, dy):
    image = image.reshape([28, 28])
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

In [ ]:
image = train_image.loc[1000].to_numpy()
shifted_image_down = shift_image(image, 0, 5)
shifted_image_left = shift_image(image, -5, 0)

plt.figure(figsize=(12,3))
plt.subplot(131)
plt.title("Original", fontsize=14)
plt.imshow(image.reshape(28, 28), interpolation="nearest", cmap="Greys")
plt.subplot(132)
plt.title("Shifted down", fontsize=14)
plt.imshow(shifted_image_down.reshape(28, 28), interpolation="nearest", cmap="Greys")
plt.subplot(133)
plt.title("Shifted left", fontsize=14)
plt.imshow(shifted_image_left.reshape(28, 28), interpolation="nearest", cmap="Greys")
plt.show()

In [ ]:
train_image = train_image_small.to_numpy()
train_target = train_target_small.to_numpy()

In [ ]:
train_image_augmented = [image for image in train_image]
train_target_augmented = [label for label in train_target]

for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    for image, label in zip(train_image, train_target):
        train_image_augmented.append(shift_image(image, dx, dy))
        train_target_augmented.append(label)

train_image_augmented = np.array(train_image_augmented)
train_target_augmented = np.array(train_target_augmented)

In [ ]:
# Shuffle the dataset
shuffle_idx = np.random.permutation(len(train_image_augmented))
train_image_augmented = train_image_augmented[shuffle_idx]
train_target_augmented = train_target_augmented[shuffle_idx]

In [ ]:
# model3 = LogisticRegression(multi_class='ovr')
model3.fit(train_image_augmented, train_target_augmented)

In [ ]:
test_prediction = model3.predict(test_image)
confusion_matrix(test_target, test_prediction)

## Build a Neural Network Model Using Tensorflow.keras

Building the neural network requires **configuring the layers** of the model, then **compiling** the model.

**Configure the layers:**

- Use `keras.layers.Sequential()` to build the network. Specify its layers as a list. Layers may include:
- Use `keras.layers.Flatten()` as the input layer. Specify `input_shape`.
- Use `keras.layers.Dense()` as hidden layers. Specify layer size and use `tf.nn.relu` as activation function.
- Use `keras.layers.Dense()` as output layers. **What is the size of the output layer?** Use `tf.nn.softmax` as activation function.

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

In [ ]:
model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(100, activation=tf.nn.relu),
    keras.layers.Dense(50, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

**Compile the model:** Specify the following parameters:

- `optimizer`: The method to minimize the loss function.
- `loss`: The loss function
- `metrics`: Measurement of model performance

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Train the model

Training the neural network model requires the following steps:

- Feed the training data to the model—in this example, the train_images and train_labels arrays.
- The model learns to associate images and labels.
- We ask the model to make predictions about a test set—in this example, the test_images array. We verify that the predictions match the labels from the test_labels array.

In [ ]:
# use the fit() method to train the model. Specify epochs.
model.fit(train_image, train_target, epochs=5)

In [ ]:
# Evaluate the model accuracy
ind = 321
image = test_image.loc[ind].to_numpy()
plt.imshow(image.reshape(28, 28), cmap=matplotlib.cm.binary)

In [ ]:
# Make predictions
probs = model.predict(test_image.loc[ind].to_numpy())
probs